In [1]:
import pandas as pd
import numpy as np
import pygad
import random
import copy

In [2]:
import csv

# Initialize an empty dictionary
enginge_RUL = {}

with open('RUL_consultancy_predictions_A3-2.csv', newline='') as csvfile:
    # Create a CSV reader object with semicolon as delimiter
    reader = csv.reader(csvfile, delimiter=';')
    # Skip the header row
    next(reader)
    # Loop over each row in the CSV
    for row in reader:
        # Check if row has exactly 2 columns
        if len(row) == 2:
            enginge_RUL[int(row[1])] = int(row[0])

In [3]:
engine_RUL_30 = {key: value for key, value in enginge_RUL.items() if value <= 30}

{20: 10,
 24: 26,
 31: 14,
 34: 8,
 35: 8,
 36: 24,
 37: 21,
 40: 29,
 41: 23,
 42: 13,
 49: 14,
 56: 18,
 61: 23,
 64: 24,
 66: 18,
 68: 13,
 76: 3,
 77: 27,
 81: 6,
 82: 11,
 90: 28,
 91: 29,
 92: 24,
 100: 24}

In [4]:
Team_A_time = {key: 4 for key in range(1, 21)}
Team_A_time.update({key: 3 for key in range(21, 56)})
Team_A_time.update({key: 2 for key in range(56, 81)})
Team_A_time.update({key: 8 for key in range(81, 101)})

Team_B_time = {j : Team_A_time[j] + 1 for j in range(1, 26)}
Team_B_time.update({j : Team_A_time[j] + 2 for j in range(26, 71)})
Team_B_time.update({j : Team_A_time[j] + 1 for j in range(71, 101)})

In [5]:
T1_time = Team_A_time
T2_time = Team_B_time
T3_time = Team_A_time
T4_time = Team_B_time

# Random Initializer

In [25]:
def calculate_team_work_time(team, engines):
    if team in ["T1", "T3"]:
        engine_times = Team_A_time
    else:
        engine_times = Team_B_time
    
    time = 0
    
    for engine in engines:
        time += engine_times[engine]
    
    return time
    

def random_init(engines_RUL):
    schedule = {"T1":[], "T2":[], "T3":[], "T4":[]}
    
    possible_engines = list(engines_RUL.keys())
    possible_teams = ["T1", "T2", "T3", "T4"]
    
    iterations = 0 
    
    while possible_engines and iterations < 500:
        index = random.randrange(4)
        team = possible_teams[index]
        
        index = random.randrange(len(possible_engines))
        selected_engine = possible_engines[index]
        
        schedule[team].append(selected_engine)
        
        possible_engines.remove(selected_engine)
        
        iterations += 1

    return schedule

In [26]:
random_init(engine_RUL_30)

{'T1': [77, 31, 42, 100, 36],
 'T2': [76, 91, 49, 34, 24, 37, 35, 90, 82, 68],
 'T3': [61, 64, 56],
 'T4': [66, 41, 20, 81, 40, 92]}

# Loss Function

In [19]:
def engine_fix_date(schedule):
    engine_fix = {}
    for team in schedule:
        days = 0
        if team == "T1" or team == "T3":
            time_table = Team_A_time
        else:
            time_table = Team_B_time
            
        for engine in schedule[team]:
            engine_fix[engine] = days
            days = days + time_table[engine]
              
    return engine_fix


initial_schedule = random_init(engine_RUL_30)
print(initial_schedule)

engine_cost = {key: 4 for key in range(1, 21)}
engine_cost.update({key: 3 for key in range(21, 31)})
engine_cost.update({key: 2 for key in range(31, 46)})
engine_cost.update({key: 5 for key in range(46, 81)})
engine_cost.update({key: 6 for key in range(81, 100)})


def loss_function(schedule, engine_RUL, engine_cost):
    loss = 0
    fix_dates = engine_fix_date(schedule)
    print(fix_dates)
    print(engine_RUL)
    for team in schedule:
        for engine in schedule[team]:

            days_late = fix_dates[engine] - engine_RUL[engine]
            engine_loss = 0
            if days_late > 0:
                cost = engine_cost[engine]
    
                for i in range(days_late):
                    day_loss = cost * (i ** 2)
                    if day_loss > 250:
                        day_loss = 250
                    engine_loss += day_loss
            loss += engine_loss
        
    return loss  

loss_schedule = loss_function(initial_schedule, engine_RUL_30, engine_cost)
print(loss_schedule)

{'T1': [37, 41, 82, 40, 91, 56, 49], 'T2': [64, 77, 35, 20, 92], 'T3': [90, 68, 36, 34, 24, 42, 100], 'T4': [81, 76, 61, 31, 66]}
{37: 0, 41: 3, 82: 6, 40: 14, 91: 17, 56: 25, 49: 27, 64: 0, 77: 4, 35: 7, 20: 12, 92: 17, 90: 0, 68: 8, 36: 10, 34: 13, 24: 16, 42: 19, 100: 22, 81: 0, 76: 9, 61: 12, 31: 16, 66: 21}
{20: 10, 24: 26, 31: 14, 34: 8, 35: 8, 36: 24, 37: 21, 40: 29, 41: 23, 42: 13, 49: 14, 56: 18, 61: 23, 64: 24, 66: 18, 68: 13, 76: 3, 77: 27, 81: 6, 82: 11, 90: 28, 91: 29, 92: 24, 100: 24}
2881


# Termination Criterion

In [ ]:
# TO DO LATER

# Selection of parents

In [20]:
# TO DO SELECT ONE DROM PYGAD UTILS
# https://pygad.readthedocs.io/en/latest/pygad.html#parent-selection-methods

# Crossover

In [66]:
def crossover(parents, offspring_size = 2):
    parent_1 = parents[0]
    parent_2 = parents[1]
    
    child1 = {key: [] for key in parent_1}
    child2 = {key: [] for key in parent_2} 
    cross_side = random.choice(["Left", "Right"]) # Right means you keep the right side of p1 and switch the right side of p2
    print(cross_side)
    for key in parent_1:
        p1_middle = len(parent_1) // 2
        p2_middle = len(parent_2) // 2
        
        if cross_side == "Left":
            sub_schedule_p1 = parent_1[key][:p1_middle]
            sub_schedule_p2 = parent_2[key][:p2_middle]
            
            child1[key].extend(sub_schedule_p2)
            child1[key].extend(parent_1[key][p1_middle:])
            
            child2[key].extend(sub_schedule_p1)
            child2[key].extend(parent_2[key][p2_middle:])
        
        if cross_side == "Right":
            sub_schedule_p1 = parent_1[key][p1_middle:]
            sub_schedule_p2 = parent_2[key][p2_middle:]
            
            child1[key].extend(parent_1[key][:p1_middle])
            child1[key].extend(sub_schedule_p2)
            
            child2[key].extend(parent_2[key][:p2_middle])
            child2[key].extend(sub_schedule_p1)
            
            
    return child1, child2
        
        
        
        
    
    

In [40]:
parent_1 = random_init(engine_RUL_30)

child1 = {key: [] for key in parent_1}
child1['T1'].extend([1,2,3])
child1['T1'].extend([4,5,6])
child1

{'T1': [1, 2, 3, 4, 5, 6], 'T2': [], 'T3': [], 'T4': []}

In [42]:
parent_1 = random_init(engine_RUL_30)
parent_2 = random_init(engine_RUL_30)

In [67]:
print(parent_1)
print(parent_2)

crossover(parents = [parent_1, parent_2])

{'T1': [90, 76, 100, 20], 'T2': [61, 81, 77, 37], 'T3': [35, 36, 31, 56, 49, 24, 66], 'T4': [41, 34, 92, 42, 40, 68, 82, 64, 91]}
{'T1': [91, 31, 24, 37, 20, 90], 'T2': [49, 34, 41, 77, 61], 'T3': [64, 100, 35, 36, 81, 66, 40, 56, 92, 76, 82], 'T4': [68, 42]}
Left


({'T1': [91, 31, 100, 20],
  'T2': [49, 34, 77, 37],
  'T3': [64, 100, 31, 56, 49, 24, 66],
  'T4': [68, 42, 92, 42, 40, 68, 82, 64, 91]},
 {'T1': [90, 76, 24, 37, 20, 90],
  'T2': [61, 81, 41, 77, 61],
  'T3': [35, 36, 35, 36, 81, 66, 40, 56, 92, 76, 82],
  'T4': [41, 34]})